### Scraper for <a href="https://xn--mckucyg8a4337c8em.com/category/column"> 新潟ガーデン (Nigata Garden) </a> website

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests

In [2]:
def get_chrome_Driver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    return driver

def get_soup(URL):
    r = requests.get(URL)
    if r:
        driver = get_chrome_Driver()
        driver.get(URL)
        htmlContent = driver.page_source
        soup =  BeautifulSoup(htmlContent, 'html.parser')
        driver.close()
        return soup
    else:
        print("Error Fetching Soup")
        return False

def get_link_from_soup(soup):

    link_list = set()
    try:
        wanted_div = soup.findAll("ul", {"class":"p-postList -type-card -pc-col3 -sp-col2"})[0]
    except IndexError:
        print("HTML element not found, returning empty set")
        return set()
    for div in wanted_div.findAll("li", {"class":"p-postList__item"}):
        wanted_article = div.find('a', href=True)["href"]
        link_list.add(wanted_article)
    return link_list

def save_link_as_txt(link_list):
    file_path = "links/nigata-garden.txt"
    # if file exist, get its content, merge to existing set and then overwrite
    if os.path.isfile(file_path):
        with open(file_path, 'r+', encoding="utf-8") as file:
            # get existing links in file
            old_links = set(file.readlines())
            # remove duplicates 
            link_list = link_list - old_links
            file.seek(0)
            for i in link_list:
                file.write(i+"\n")
            file.truncate()
    else: # if not, make new file
        with open(file_path, "w", encoding="utf-8") as file:
            for i in link_list:
                file.write(i+"\n")
    return True

In [3]:
all_links = set()
for i in range(1,100):
    soup = get_soup(f"https://xn--mckucyg8a4337c8em.com/category/column/page/{i}")
    if soup:
        current_links = get_link_from_soup(soup)
        if len(current_links) > 0: # continue until no tags are found 
            all_links.update(get_link_from_soup(soup))
        else: # if tags we want are not found
            break
    else: # if encountered HTML error such as 404
        break

save_link_as_txt(all_links)

HTML element not found, returning empty set


True